In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
import pickle
%matplotlib inline
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from xgboost import XGBRegressor

In [2]:
fang=pd.read_csv('batting_all.csv')

In [140]:
fang['Name']=' '+fang['Name']

In [141]:
[i for i in range(2015,2020)]

[2015, 2016, 2017, 2018, 2019]

In [143]:
predseasons=['exitv'+str(i) for i in range(2015,2020)]
i=2015
for season in predseasons:
  
    globals()[season] = pd.read_csv('exit_velocity'+str(i)+'.csv')
    globals()[season]['Name']=globals()[season][' first_name']+' '+globals()[season]['last_name']
    i+=1

In [144]:
predseasons=['sprint'+str(i) for i in range(2015,2020)]
i=2015
for season in predseasons:
  
    globals()[season] = pd.read_csv('sprint_speed'+str(i)+'.csv')
    globals()[season]['Name']=globals()[season][' first_name']+' '+globals()[season]['last_name']
    i+=1

In [3]:
predseasons=['stats'+str(i)+str(i+1) for i in range(2002,2019)]
i=2002
for season in predseasons:
  
    globals()[season] = pd.merge(fang[fang['Season']==i],fang[fang['Season']==(i+1)][['Name','wOBA']],on="Name")
    i+=1

In [146]:
predseasons=['stats'+str(i)+str(i+1) for i in range(2015,2019)]
i=2015
for season in predseasons:
  
    globals()[season] = pd.merge(globals()[season],globals()['sprint'+str(i)][['Name','sprint_speed']],on="Name")
    i+=1

In [147]:
predseasons=['stats'+str(i)+str(i+1) for i in range(2015,2019)]
i=2015
for season in predseasons:
  
    globals()[season] = pd.merge(globals()[season],globals()['exitv'+str(i)][['avg_hit_angle',
       'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance','ev95percent', 'brl_percent', 'brl_pa', 'Name']],on="Name")
    i+=1

In [127]:
current=pd.read_csv('batting2019.csv')
current['Name']=' '+current['Name']
current=pd.merge(current,sprint2019[['Name','sprint_speed']],on="Name")
current=pd.merge(current,exitv2019[['avg_hit_angle',
      'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance','ev95percent', 'brl_percent', 'brl_pa', 'Name']],on="Name")

In [6]:
statsrel=pd.concat([globals()[season] for season in predseasons])

In [7]:
statsrel=statsrel.rename(columns={'wOBA_x':'wOBA_current_year','wOBA_y':'wOBA_next_year'})
#current=current.rename(columns={'wOBA':'wOBA_current_year'})
#s2018=s2018.rename(columns={'wOBA':'wOBA_current_year'})


In [8]:
statsrel=statsrel.reset_index(drop=True)
#current=current.reset_index(drop=True)
#s2018=s2018.reset_index(drop=True)


In [9]:
for col in statsrel.columns:
    if type(statsrel[col][1])==str:
        statsrel[col]=statsrel[col].str.replace('%','',regex=False)
        
#for col in current.columns:
 #   if type(current[col][1])==str:
  #      current[col]=current[col].str.replace('%','',regex=False)
        
        
#for col in s2018.columns:
 #   if type(s2018[col][1])==str:
  #      s2018[col]=s2018[col].str.replace('%','',regex=False)
        


In [ ]:
'sprint_speed', 'avg_hit_angle',
       'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance', 'ev95percent',
       'brl_percent', 'brl_pa']

In [10]:
numbers=['Age', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP',
       'SLG', 'OPS', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%',
       'HR/FB', 'IFH%', 'BUH%', 'wOBA_current_year', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
       'wOBA_next_year']
for col in numbers:
    statsrel[col]=statsrel[col].astype(float)
    
    

In [55]:
numbers=['Age', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP',
       'SLG', 'OPS', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%',
       'HR/FB', 'IFH%', 'BUH%', 'wOBA_current_year', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%', 'sprint_speed','avg_hit_angle',
       'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance', 'ev95percent',
       'brl_percent', 'brl_pa']
for col in numbers:
    current[col]=current[col].astype(float)
    s2018[col]=s2018[col].astype(float)
    
    

ValueError: could not convert string to float: '13.3 %'

In [ ]:
'avg_hit_angle',
       'anglesweetspotpercent', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance', 'ev95percent',
       'brl_percent', 'brl_pa','sprint_speed']

In [11]:
features=['AVG', 'BB%', 'K%', 'BB/K', 'OBP',
       'SLG', 'ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%',
       'HR/FB', 'IFH%', 'BUH%', 'wOBA_current_year', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%']

In [154]:
featureswithestimators=['Age', 'AVG', 'BB%', 'K%', 'BB/K', 'OBP',
       'SLG','ISO', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%',
       'HR/FB', 'IFH%', 'BUH%', 'wOBA_current_year', 'O-Swing%', 'Z-Swing%',
       'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%',
       'SwStr%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%', 'sprint_speed', 'avg_hit_angle',
       'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'fbld', 'gb',
       'max_distance', 'avg_distance', 'avg_hr_distance', 'ev95percent',
       'brl_percent', 'brl_pa']

In [12]:
corrdict2={col:np.abs(statsrel[col].corr(statsrel['wOBA_next_year'])) for col in featureswithestimators}



    
Corr=pd.DataFrame.from_dict(corrdict2, orient='index')
Corr.sort_values(by=0,ascending=False)

NameError: name 'featureswithestimators' is not defined

In [13]:
#current=current.dropna()
statsrel=statsrel.dropna()
#s2018=s2018.dropna()

In [108]:
from sklearn.preprocessing import PolynomialFeatures
#poly = PolynomialFeatures(2,interaction_only=True)
X=poly.fit_transform(statsrel[features])

In [473]:
from sklearn.model_selection import train_test_split

In [ ]:
train_test_split

In [14]:
X=statsrel[features]
y=statsrel['wOBA_next_year']

In [15]:
scaler=StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(statsrel[features],statsrel['wOBA_next_year'])
X_train.shape


(1224, 32)

In [16]:
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
X_train,y_train=shuffle(X_train,y_train,random_state=1)
X,y=shuffle(X,y)

In [18]:
import sklearn
MAEwOBA = sklearn.metrics.mean_squared_error(y_true = statsrel.wOBA_next_year, y_pred = statsrel.wOBA_current_year)
np.sqrt(MAEwOBA)

0.033132691714637307

In [ ]:
pipeline.

In [23]:
regressor = LinearRegression()
pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])

E=np.mean(cross_val_score(pipeline,X,y,scoring='neg_mean_squared_error'))
(-E)**(.5)
#pipeline.fit(X_train,y_train)
#MAE=sklearn.metrics.mean_absolute_error(y_true = y_test, y_pred = pipeline.predict(X_test))

#MAE

0.028480095318555318

In [66]:
from sklearn.linear_model import Ridge

regressor=Ridge(alpha=1)
pipeline=Pipeline([('scaler',MinMaxScaler()),('regressor',regressor)])
E=np.mean(cross_val_score(pipeline,X,y,scoring='neg_mean_squared_error'))
(-E)**(.5)
#pipeline.fit(X_train,y_train)
#MAE=sklearn.metrics.mean_absolute_error(y_true = y_test, y_pred = pipeline.predict(X_test))
#MAE

0.02842631976963471

In [39]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso

regressor=Lasso(alpha=.00001)
pipeline=Pipeline([('scaler',MinMaxScaler()),('regressor',regressor)])
E=np.mean(cross_val_score(pipeline,X,y,scoring='neg_mean_squared_error'))
(-E)**(.5)
#pipeline.fit(X_train,y_train)
#MAE=sklearn.metrics.mean_absolute_error(y_true = y_test, y_pred = pipeline.predict(X_test))
#MAE

0.028444699142681024

In [63]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet

regressor=ElasticNet(alpha=0.05,l1_ratio=.005)
pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])
E=np.mean(cross_val_score(pipeline,X,y,scoring='neg_mean_squared_error'))
(-E)**(.5)
#pipeline.fit(X_train,y_train)
#MAE=sklearn.metrics.mean_absolute_error(y_true = y_test, y_pred = pipeline.predict(X_test))
#MAE

0.028456195146985528

In [79]:
import scipy.stats as st
regressor=XGBRegressor(booster='gbtree',n_estimators=10000,max_depth=1,learning_rate=0.06,
                       reg_lambda=0,reg_alpha=.01,gamma=0.08,min_child_weight=25)

pipeline=Pipeline([('scaler',scaler),('regressor',regressor)])
parameters = {'regressor__n_estimators': st.randint(50,300),
             'regressor__max_depth':st.randint(1,10),
             'regressor__learning_rate':st.expon(scale=0.05),
             'regressor__reg_alpha':st.expon(scale=.1),
              'regressor__gamma':st.expon(scale=0.08),
             'regressor__min_child_weight':st.randint(1,50)}
model = RandomizedSearchCV(pipeline,parameters,n_iter=1000, cv=5, iid=False, n_jobs=-1,refit=True,verbose=5,scoring='neg_mean_squared_error') 
model.fit(X,y)
#pipeline.fit(X,y)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   58.9s


KeyboardInterrupt: 

In [ ]:
(-1*(model.best_score_))**(.5)

In [740]:
print('Best Parameters: ' , model.best_params_)
print('\n')
print('Best Score: ' ,  -1*model.best_score_)

Best Parameters:  {'regressor__gamma': 0.001104734172276005, 'regressor__learning_rate': 0.07657368027243552, 'regressor__max_depth': 2, 'regressor__min_child_weight': 33, 'regressor__n_estimators': 158, 'regressor__reg_alpha': 0.14210018311065148}


Best Score:  0.022971380197701046


In [70]:
regressor=Ridge(alpha=1)
pipeline=Pipeline([('scaler',MinMaxScaler()),('regressor',regressor)])
np.sort(np.abs(pipeline.fit(X,y).named_steps['regressor'].coef_))

array([0.00115336, 0.00130179, 0.00158224, 0.00246981, 0.00297171,
       0.00352811, 0.00361635, 0.00380728, 0.00526774, 0.00651082,
       0.00713478, 0.00818979, 0.00856888, 0.00977071, 0.01055603,
       0.01113103, 0.0139092 , 0.01401205, 0.01480058, 0.01576447,
       0.0158275 , 0.01878629, 0.01983721, 0.02261362, 0.02440596,
       0.0244398 , 0.02709145, 0.02888303, 0.03209588, 0.03289364,
       0.04029015, 0.04105373])

In [74]:
most=np.argsort(np.abs(pipeline.fit(X,y).named_steps['regressor'].coef_))[:]
for i in reversed([statsrel[features].columns[i] for i in most]):
    print(i)

ISO
BB%
SLG
OBP
Hard%
BB/K
SwStr%
Soft%
O-Contact%
Oppo%
Med%
IFH%
Contact%
BABIP
Swing%
wOBA_current_year
O-Swing%
Cent%
GB/FB
Pull%
Z-Swing%
HR/FB
K%
F-Strike%
IFFB%
AVG
GB%
Zone%
FB%
Z-Contact%
LD%
BUH%


In [176]:
XC = StandardScaler().fit_transform(current[features])

In [163]:
pd.set_option('display.max_rows', 500)

In [177]:
regressor=Ridge(alpha=200)
regressor.fit(X,y)
regressor.predict(XC)
current['preds']=regressor.predict(XC)

In [180]:
preds=current[['Name','wOBA_current_year','preds']].sort_values('preds',ascending=False)
preds

,Name,wOBA_current_year,preds
0,Cody Bellinger,0.496,0.413961
8,Mike Trout,0.418,0.410328
3,Josh Bell,0.449,0.402960
6,Joey Gallo,0.430,0.394255
4,Anthony Rendon,0.439,0.394057
2,Christian Yelich,0.449,0.391128
75,Ronald Acuna Jr.,0.357,0.386932
1,Austin Meadows,0.461,0.386078
49,Mookie Betts,0.372,0.382733
15,Hunter Dozier,0.409,0.382211


In [181]:
preds.to_csv('leaders2019.csv')

In [40]:
regressor=Ridge(alpha=100)
regressor.fit(X,y)
regressor.predict(X)
statsrel['preds']=regressor.predict(X)
statsrel['preds'].corr(statsrel['wOBA_next_year'])

0.6652568120587895

In [80]:

regressor=Ridge(alpha=1)
pipeline=Pipeline([('scaler',MinMaxScaler()),('regressor',regressor)])
pipeline.fit(X,y)
pickle.dump(pipeline,open('2020_woba_model.pkl','wb'))

In [683]:
regressor

LinearRegression()

In [732]:
pipeline

Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Ridge(alpha=30))])

In [81]:
pick2=open('2020_woba_model.pkl','rb')
wobareg=pickle.load(pick2)


In [82]:
wobareg

Pipeline(steps=[('scaler', MinMaxScaler()), ('regressor', Ridge(alpha=1))])